In [ ]:
import pandas as pd
import spacy
import string
import regex as re
import nltk
import ast
import copy
import glob
# import advertools as adv
import plotly.graph_objects as go

from datetime import datetime


pd.options.display.max_colwidth = 285



from textblob import TextBlob

# from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer


from spacymoji import Emoji
from nltk.corpus import stopwords
from urllib.parse import urlparse
from textblob import Word
# from sklearn.metrics.pairwise import cosine_similarity



# nlp = spacy.load("en_core_web_sm")
# emoji = Emoji(nlp)
# nlp.add_pipe("emoji", first=True)


# stemmer = nltk.SnowballStemmer("english")

In [ ]:
%run ./modules/load_data_module.ipynb
%run ./modules/extract_info_clean_data_module.ipynb
%run ./modules/update_dataframe_module.ipynb
# %run ./modules/sentiment_module.ipynb
# %run ./modules/semantic_similarity_module.ipynb

In [ ]:
sample_tweets = pd.read_feather('/home/niloofar/Developement/Climate-Change/data/sample_data/sample_tweets_final_version.feather')
sample_replies = pd.read_feather('/home/niloofar/Developement/Climate-Change/data/sample_data/sample_replies_final_version.feather')
sample_quotes = pd.read_feather('/home/niloofar/Developement/Climate-Change/data/sample_data/sample_quotes_final_version.feather')

In [ ]:
def split_dataframe(df, chunk_size = 10000): 
    list_of_chunks = []
    num_chunks = len(df) // chunk_size + 1
    for i in range(num_chunks):
        list_of_chunks.append(df[i*chunk_size:(i+1)*chunk_size])
    return list_of_chunks

In [ ]:
def restrict_df_to_time_interval(df, from_day, to_day):

    df['created_at'] = df['created_at'].apply(lambda x: datetime.strptime(x, '%Y-%m-%dT%H:%M:%S.%fZ'))
    return df.loc[(df['created_at']>= from_day) & (df['created_at'] < to_day)]

In [ ]:
def update_controversiality_score(df_tweet, df_reply, df_quote):# from_idx, to_idx):
    """
    input:
        df_tweet: sample dataframe of tweets
        df_reply: sample dataframe of replies
        df_quote: sample dataframe of quotes
        from_idx: integer, the first index of the tweets sub-sample. 
        to_idx: integer, the last index of the tweets sub-sample

    output:
        dataframe, the tweets dataframe, the GS_controversiality_score of the selected sub-sample is updated.
    
    How it works?
        it shows a tweet and all information about it as a dataframe
        it then asks if you want to see the dataframe contining the replies and the information about replies
        TO SEE THE REPLIES: press y, to continue without reading the replies, press any other key
        it ask it you want to see the dataframe containing the quotes
        TO SEE THE QUOTES: press y, to continue without reading the replies press any other key
        
        Finally:
            it asks you to give some score for the GS_controversiality_score. the inputs are:
                0 : not controversial
                1 : somewhat controversial
                2 : controversial
                3 : very controversial
    """
    # df_sample = copy.deepcopy(df_tweet.iloc[from_idx:to_idx])
    df_sample = copy.deepcopy(df_tweet)
    
    
    for i, row in df_sample.iterrows():
        display(row.to_frame())
        
        print('Read replies?')
        reply = input()
        
        if reply == 'y':
            display(get_conversation(row['conversation_id'], df_reply))
            
        print('Read quotes?')
        quote = input()
        
        if quote == 'y':
            display(get_conversation(row['conversation_id'], df_quote))
            
            
        print('The score would be:')
        score_range = [0, 1, 2, 3]
        score = input()
        
        if int(score) not in score_range:
            print('INVALID SCORE')
            score = input()
            if int(score) not in score_range:
                print("FINAL CHANCE TO GIVE A VALID SCORE!")
                score = input()
        
        
        df_tweet.at[i, ['GS_controversiality_score']] = score
        
        print(f"score stored for the {i} row is {score}")
        
            
        print('********************************************************************************************', end="")
       
        
    return df_tweet
        
        


# Scoring :D

In [ ]:
# Toy example of the function
# df = update_controversiality_score(sample_tweets, sample_replies, sample_quotes, 500, 1000)


 
<span style="color:red"> **To save the scores use  df.to_csv('name.csv', index=False)**</span>